In [2]:
%serialconnect

Connecting to Serial /dev/ttyUSB0 baud=115200

 ** Serial connected **

Serial<id=0x7f0e4164ad30, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)

>>> 
>>> 
>>> 
>>> 
raw REPL; CTRL-B to exit
>

In [21]:
import network, time, socket

si = network.WLAN(network.STA_IF)
si.active(True)
#assert not si.isconnected()

knownwifis = dict(k.split(b":")[:2]  for k in open("wificodes.txt", "rb"))
wifiname = max((sc  for sc in si.scan()  if sc[0] in knownwifis), key=lambda X: X[3])[0]  # X[3]=RSSI (received signal strength)
print("strongest known DoESWifi signal", wifiname)
si.connect(wifiname, knownwifis[wifiname])
del knownwifis

while not si.isconnected():
    time.sleep(0.1)
    pass

ipnumber = si.ifconfig()[0]
print("Connect through wifi to", ipnumber)


I (1237491) wifi: event 1
strongest known DoESWifi signal b'The Sensible Code Company'
Connect through wifi to 192.168.1.33


In [15]:
.

[leftinbuffer] I (965201) wifi: pm start, type:0
[leftinbuffer] 
Traceback (most recent call last):
  File "<stdin>", line 1
SyntaxError: invalid syntax


In [16]:
assert si.isconnected()

s = socket.socket()
portnumber = 9997

s.bind(socket.getaddrinfo("0.0.0.0", portnumber)[0][-1])  # recommended to always make addresses via getaddrinfo()
s.listen(1)  # no backlog of connections allowed

print("\nPlease connect to ({}, {})".format(ipnumber, portnumber))
print("  mac-address {}".format(":".join(hex(x)[-2:]  for x in si.config("mac"))))
espname = "ESP_{}".format("".join(hex(x)[-2:]  for x in si.config("mac")[-3:]))



Please connect to (192.168.1.33, 9997)
  mac-address 30:ae:a4:80:78:20


In [17]:
print("Waiting for a connection on ({}, {})".format(ipnumber, portnumber))
ss, addr = s.accept()
print("connection made incoming from", addr)

Waiting for a connection on (192.168.1.33, 9997)
.......connection made incoming from ('192.168.1.31', 54586)


In [ ]:
# You can now read and write to this connection 
# by iterating between the next two cells

In [18]:
# Send something out to the other end (for it to read)
ss.write("Hi-{} from {}\n".format(time.time(), espname))

In [19]:
# Read a line written from the other end 
print(ss.readline())

b'Hello-1506940264 from PC\n'


In [20]:
ss.close()